## AutoLamella Exports
Exportable statistics for each experiment, including milling parameters, timing, and success rates.

In [ ]:
%load_ext autoreload
%autoreload 2

import glob
import os
from pprint import pprint
import matplotlib.pyplot as plt
import pandas as pd

from fibsem.milling.patterning.plotting import draw_milling_patterns
from fibsem.milling import get_milling_stages
from fibsem.structures import FibsemImage, Point

from autolamella.structures import Experiment
from autolamella.tools.data import calculate_statistics_dataframe


pd.set_option("display.max_rows", 50)


In [ ]:
PATH = "/home/patrick/data/monash-cryo-em/AutoLamella-Exports" 

filenames = glob.glob(os.path.join(PATH, "**/experiment.yaml"), recursive=True)

pprint(filenames)


In [ ]:
from autolamella.structures import AutoLamellaStage

for filename in filenames:
    experiment_path = os.path.dirname(filename)
    print(f"Experiment: {experiment_path}")

    try:
        exp = Experiment.load(os.path.join(experiment_path, "experiment.yaml"))

        print(f"Total Lamella: {len(exp.positions)}, Finished Lamella: {len(exp.at_stage(AutoLamellaStage.Finished))})")
        failed_lamella = exp.at_failure()
        print(f"Failed Lamella: {[l.name for l in failed_lamella]}")

        df = exp._create_protocol_dataframe()
        # drop spacing, rate, preset, spot_size #Tescan only
        TESCAN_PARAMS = ["spacing", "rate", "preset", "spot_size"]
        df = df.drop(columns=TESCAN_PARAMS)

        # filter to WorkflowStage == "MillRough", "MillPolishing", fiducial
        milling_workflows = ["MillRoughCut", "MillPolishingCUt", "microexpansion", "fiducial"]
        df = df[df["WorkflowStage"].isin(milling_workflows)]

        # filter to only milling_current, voltage, depth
        df = df[["Experiment", "Lamella", "WorkflowStage", "MillingStage", 
                 "type", "milling_current", "milling_voltage", "depth", "lamella_height", "lamella_width", "height", "width"]]
        display(df)

        # save to csv at exp.path "milling.csv"
        df.to_csv(os.path.join(exp.path, "protocol.csv"), index=False)

        # continue
        for pos in exp.positions:
            
            if not pos.is_failure:
                continue
            print(f"{pos.name}: {pos.failure_note}")
            
            # load milling stages
            protocol = pos.protocol
            milling_stages = []
            for mw in milling_workflows:
                stages = get_milling_stages(key=mw, protocol=protocol, point=Point.from_dict(protocol[mw]["point"]))
                milling_stages.extend(stages)

            # TODO: lamella path is not correct when re-loaded on another machine
            fib_image = FibsemImage.load(os.path.join(exp.path, pos.name, "ref_MillPolishingCut_final_high_res_ib.tif"))
            sem_image = FibsemImage.load(os.path.join(exp.path, pos.name, "ref_MillPolishingCut_final_high_res_eb.tif"))

            fig, ax1 = draw_milling_patterns(fib_image, milling_stages)
            plt.title(pos.name)
            plt.show()

            fig, ax = plt.subplots(1, 2, figsize=(10, 5))
            ax[0].imshow(sem_image.data, cmap="gray")
            ax[1].imshow(fib_image.data, cmap="gray")

            plt.show()

            print("-"*80)
            continue
    except Exception as e:
        print(e)
        continue
    

In [ ]:
exp = Experiment.load(os.path.join(experiment_path, "experiment.yaml"))

df = exp._create_protocol_dataframe()

display(df)

for s,v in exp.positions[0].states.items():
    print(s, v.completed)


### Protocol Up-Convert


In [ ]:
%load_ext autoreload 
%autoreload 2

from autolamella.protocol.validation import validate_protocol, validate_and_convert_protocol, convert_old_milling_protocol_to_new_protocol
from autolamella.structures import AutoLamellaProtocol
from fibsem import utils
from pprint import pprint 


PROTOCOL_PATH = "/home/patrick/data/monash-cryo-em/AutoLamella-Exports/autolamella/20241010_Sai/AutoLamella-2024-10-10-09-07/protocol.yaml"
# protocol = AutoLamellaProtocol.load(PROTOCOL_PATH)

ddict = utils.load_protocol(PROTOCOL_PATH)

# pprint(ddict["milling"])


ddict2 = validate_protocol(ddict)

# pprint(ddict2["milling"])


mdict = convert_old_milling_protocol_to_new_protocol(ddict["milling"])

pprint(mdict)


### Overview Position Export

In [ ]:
%load_ext autoreload
%autoreload 2

from fibsem import utils
from autolamella.structures import Experiment, AutoLamellaStage
from fibsem.structures import FibsemImage, Point
from fibsem.imaging.tiled import plot_stage_positions_on_image
%matplotlib inline
import matplotlib.pyplot as plt
import os
from pprint import pprint

experiment_path = "/home/patrick/development/openfibsem/autolamella/autolamella/log/AutoLamella-2025-03-04-18-03/"
filename = os.path.join(experiment_path, "overview-image.tif")
image = FibsemImage.load(filename)

exp = Experiment.load(os.path.join(experiment_path, "experiment.yaml"))

from autolamella.tools.reporting import  save_final_overview_image

fig = save_final_overview_image(exp, 
                          image, 
                          os.path.join(experiment_path, "final-overview-image.png"))

plt.show()